# Results

In [14]:
from pathlib import Path
import pandas as pd
from src.helpers.metrics import  build_metrics_table

from experiment_config import DegModel,DATA_NAME,ESTIMATION_NAME,PFNET_NAME,N_REP,PRED_STAT

DATA_NAME

'DS05'

In [15]:
estimation_dir = Path('experiments')/DATA_NAME/ESTIMATION_NAME
pred_dir = estimation_dir/DegModel.name()/'pf_performs'/PFNET_NAME/f"pred_{PRED_STAT}"

## Load Predictions

In [16]:
df_preds = pd.read_csv(pred_dir / f"rul_test_predictions_rep{N_REP}.csv")

## Compute mean prediction

In [17]:
df_preds = df_preds.groupby(["unit", "time"], as_index=False).mean()

## Full-life Predictions

In [18]:
metrics_df = build_metrics_table(df_preds)
metrics_df

,unit,RMSE,Score,PICP,PINAW
0,7,7.523322,57.068116,0.976471,0.378965
1,8,7.288528,59.594713,1.000000,0.358611
2,9,2.765901,14.206868,1.000000,0.470637
3,10,6.685372,50.940912,0.975610,0.383472


In [19]:
metrics_df.drop(columns=["unit"]).mean()

RMSE      6.065781
Score    45.452652
PICP      0.988020
PINAW     0.397921
dtype: float64

## Degradation Predictions

In [20]:
hi_df = pd.read_csv(estimation_dir/'data_test.csv')
test_units = hi_df['unit'].astype(int).unique().tolist()
onsets = {unit: hi_df[(hi_df['unit']==unit) & (hi_df['hs']==0)]['cycle'].values[0] for unit in test_units}
df_degpreds = df_preds[
    df_preds["time"] > df_preds["unit"].map(onsets)
]

In [21]:
metrics_df = build_metrics_table(df_degpreds)
metrics_df

,unit,RMSE,Score,PICP,PINAW
0,7,5.243786,23.308115,0.967213,0.420970
1,8,2.578105,11.146429,1.000000,0.398438
2,9,2.012063,7.165297,1.000000,0.565038
3,10,2.667928,12.754839,0.955556,0.409842


In [22]:
metrics_df.drop(columns=["unit"]).mean()

RMSE      3.125471
Score    13.593670
PICP      0.980692
PINAW     0.448572
dtype: float64

## Benchmark Predictions

In [23]:
df_benchmark = df_preds[df_preds["true_rul"] <= 65]

In [24]:
metrics_df = build_metrics_table(df_benchmark)
metrics_df

,unit,RMSE,Score,PICP,PINAW
0,7,6.108817,31.421940,0.969697,0.413611
1,8,6.581911,35.256693,1.000000,0.399687
2,9,1.879872,9.132407,1.000000,0.477203
3,10,6.044978,36.418095,0.969697,0.399898


In [25]:
metrics_df.drop(columns=["unit"]).mean()

RMSE      5.153894
Score    28.057284
PICP      0.984848
PINAW     0.422600
dtype: float64

| DS01	 | full      | RUL65     | deg       |
|--------|-----------|-----------|-----------|
| RMSE   | 5.068149  | 4.803219  | 4.711211  |
| Score  | 52.206928 | 37.873358 | 28.425670 |
| PICP   | 0.953896  | 0.943182  | 0.920168  |
| PINAW  | 0.312042  | 0.338091  | 0.337272  |

| DS03	 | full      | RUL65     | deg       |
|--------|-----------|-----------|-----------|
| RMSE   | 7.654103  | 7.383116  | 4.767812  |
| Score  | 71.544372 | 57.373969 | 22.033015 |
| PICP   | 0.926416  | 0.909519  | 0.913396  |
| PINAW  | 0.394850  | 0.398448  | 0.403939  |

| DS05   | full      | RUL65     | deg       |
|--------|-----------|-----------|-----------|
| RMSE   | 6.065781  | 5.153894  | 3.125471  |
| Score  | 45.452652 | 28.057284 | 13.593670 |
| PICP   | 0.988020  | 0.984848  | 0.980692  |
| PINAW  | 0.397921  | 0.422600  | 0.448572  |

| DS06   | full      | RUL65     | deg	     |
|--------|-----------|-----------|-----------|
| RMSE   | 6.697169  | 5.585590  | 4.804076  |
| Score  | 71.535756 | 39.490416 | 27.586838 |
| PICP   | 0.877232  | 0.920455  | 0.940313  |
| PINAW  | 0.324644  | 0.374098  | 0.430793  |